<a href="https://colab.research.google.com/github/firarru/Text-Mining/blob/main/Sistem_rekomendasi_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# pip install --upgrade pandas

# **Loading Data**

In [3]:
import kagglehub

path = kagglehub.dataset_download("artermiloff/steam-games-dataset")
print("Path to dataset files:", path)

import os

# Cek file yang tersedia dalam dataset
files = os.listdir(path)
print("Files in dataset:", files)

# Sesuaikan dengan file yang ada - corrected file name
dataset_path = os.path.join(path, "games_march2025_cleaned.csv")
df = pd.read_csv(dataset_path)

# Tampilkan 5 baris pertama
df.head(10)

100%|██████████| 418M/418M [00:08<00:00, 52.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/artermiloff/steam-games-dataset/versions/2
Files in dataset: ['games_march2025_cleaned.csv', 'games_march2025_full.csv', 'games_may2024_full.csv', 'games_may2024_cleaned.csv']


,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608
5,440,Team Fortress 2,2007-10-10,0,0.00,1,'The most fun you can have online' - PC Gamer ...,'The most fun you can have online' - PC Gamer ...,Nine distinct classes provide a broad range of...,NaN,...,0,0,0,0,50817,"{'Free to Play': 62868, 'Hero Shooter': 61020,...",89,1146642,93,8172
6,105600,Terraria,2011-05-16,0,9.99,2,"Dig, Fight, Explore, Build: The very world is ...","Dig, Fight, Explore, Build: The very world is ...","Dig, fight, explore, build! Nothing is impossi...",NaN,...,0,0,0,0,30516,"{'Open World Survival Craft': 16365, 'Sandbox'...",97,1102434,96,8860
7,252490,Rust,2018-02-08,0,39.99,4,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,"“Rust is one of the cruelest games on Steam, a...",...,836,3675,231,0,200902,"{'Survival': 18592, 'Crafting': 11822, 'Multip...",87,993856,87,14960
8,4000,Garry's Mod,2006-11-29,0,5.99,0,Garry's Mod is a physics sandbox. There aren't...,Garry's Mod is a physics sandbox. There aren't...,Garry's Mod is a physics sandbox. There aren't...,“It brings back the silly glee of unbridled ex...,...,100,1173,136,40,32384,"{'Sandbox': 18706, 'Moddable': 14479, 'Multipl...",96,985010,96,8277
9,1172470,Apex Legends™,2020-11-04,0,0.00,0,Apex Legends: Takeover About the Game Conquer ...,"Conquer with character in Apex Legends, a free...","Apex Legends is the award-winning, free-to-pla...",“The champion of Battle Royales.” 9/10 – GameS...,...,548,667,211,0,151844,"{'Free to Play': 2170, 'Battle Royale': 1483, ...",67,983230,53,7054


In [4]:
df = df[['name', 'release_date', 'detailed_description', 'tags']]

In [5]:
df['release_year'] = pd.to_datetime(df['release_date']).dt.year

In [6]:
df.isnull().sum()/100

,0
name,0.00
release_date,0.00
detailed_description,1.97
tags,0.00
release_year,0.00


In [7]:
df.dropna(inplace=True)
df.shape

(89421, 5)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89421 entries, 0 to 89617
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  89421 non-null  object
 1   release_date          89421 non-null  object
 2   detailed_description  89421 non-null  object
 3   tags                  89421 non-null  object
 4   release_year          89421 non-null  int32 
dtypes: int32(1), object(4)
memory usage: 3.8+ MB


In [9]:
df = df[df['release_year'] > 2022].reset_index(drop=True)

In [10]:
df['content'] = df['detailed_description'] + ' ' + df['tags']

In [11]:
df = df[['name', 'release_year', 'content']]

In [12]:
df

,name,release_year,content
0,Black Myth: Wukong,2024,Digital Deluxe Edition The Black Myth: Wukong ...
1,HELLDIVERS™ 2,2024,Digital Deluxe Edition Edition includes: ‘DP-5...
2,Baldur's Gate 3,2023,Gather your party and return to the Forgotten ...
3,Lethal Company,2023,You are a contracted worker for the Company. Y...
4,Dying Light,2025,DYING LIGHT: THE BEAST Kyle Crane returns! The...
...,...,...,...
34499,Seven Spirits,2023,SEVEN SPIRITS™ is a horizontal scrolling actio...
34500,Caribbean Pirates,2024,Caribbean Pirates is a board game that immerse...
34501,Fortress Defense,2023,Prepare for an epic battle in Fortress Defense...
34502,Outrun Them,2024,The name of the game is literally the game. Ou...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34504 entries, 0 to 34503
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          34504 non-null  object
 1   release_year  34504 non-null  int32 
 2   content       34504 non-null  object
dtypes: int32(1), object(2)
memory usage: 674.0+ KB


# **Preprocessing**

In [14]:
pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


In [19]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [20]:
clean_spcl = re.compile('[/(){}\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
sastrawi = StopWordRemoverFactory()
stopworda = sastrawi.get_stop_words()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = stemmer.stem(text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

In [17]:
# df['clean_content'] = df['content'].apply(clean_text)

In [24]:
df['clean_spcl'] = df['content'].str.replace(clean_spcl, ' ')
# df['clean_symbol'] = df['clean_spcl'].apply(clean_symbol)

ValueError: Cannot use a compiled regex as replacement pattern with regex=False